## Annotation to JSON 

In [119]:
import re
regexp_entity = re.compile('(?P<variable>[A-Z][0-9]+)\s+(?P<type>[A-Z][A-Za-z_]*)\s+[0-9]+\s+[0-9]+(?P<name>(\s+[a-z]+)+$)')
regexp_attribute = re.compile('(?P<variable>[A-Z][0-9]+)\s+(?P<type>[A-Z][A-Za-z_]*)\s+(?P<target_variable>[A-Z][0-9]+)\s+(?P<value>[A-Za-z]+)')

regexp_relation = re.compile('(?P<variable>[A-Z][0-9]+)\s+(?P<referers>([A-Z][A-Za-z_]*:[A-Z][0-9]+\s*)+)')
regexp_transitive = re.compile('(?P<arity>\*)\s+(?P<type>[A-Z][A-Za-z_]*)\s+(?P<refersTo1>[A-Z][0-9]+)\s+(?P<refersTo2>[A-Z][0-9]+)\s*')

In [41]:
annotation = r"""T1	Character 0 18	patty the milkmaid
A1	Gender T1 Female
T2	Says 218 222	said
E1	Says:T2 WHAT:T3 WHO:T1 WHAT2:T4 WHAT3:T5 WHAT4:T6 WHAT5:T7 WHAT6:T8 WHAT7:T9
T3	Character_Line 179 217	i 'll buy some fowls from farmer brown
T4	Character_Line 227 302	and they will lay eggs each morning which i will sell to the parson 's wife
T5	Character_Line 303 407	with the money that i get from the sale of these eggs i 'll buy myself a new dimity frock and a chip hat
T6	Character_Line 408 480	and when i go to market wo n't all the young men come up and speak to me
T7	Character_Line 481 512	polly shaw will be that jealous
T8	Character_Line 513 530	but i do n't care
T9	Character_Line 531 582	i shall just look at her and toss my head like this"""

print(annotation)

T1	Character 0 18	patty the milkmaid
A1	Gender T1 Female
T2	Says 218 222	said
E1	Says:T2 WHAT:T3 WHO:T1 WHAT2:T4 WHAT3:T5 WHAT4:T6 WHAT5:T7 WHAT6:T8 WHAT7:T9
T3	Character_Line 179 217	i 'll buy some fowls from farmer brown
T4	Character_Line 227 302	and they will lay eggs each morning which i will sell to the parson 's wife
T5	Character_Line 303 407	with the money that i get from the sale of these eggs i 'll buy myself a new dimity frock and a chip hat
T6	Character_Line 408 480	and when i go to market wo n't all the young men come up and speak to me
T7	Character_Line 481 512	polly shaw will be that jealous
T8	Character_Line 513 530	but i do n't care
T9	Character_Line 531 582	i shall just look at her and toss my head like this


In [42]:
tups = annotation.split('\n')
tups

['T1\tCharacter 0 18\tpatty the milkmaid',
 'A1\tGender T1 Female',
 'T2\tSays 218 222\tsaid',
 'E1\tSays:T2 WHAT:T3 WHO:T1 WHAT2:T4 WHAT3:T5 WHAT4:T6 WHAT5:T7 WHAT6:T8 WHAT7:T9',
 "T3\tCharacter_Line 179 217\ti 'll buy some fowls from farmer brown",
 "T4\tCharacter_Line 227 302\tand they will lay eggs each morning which i will sell to the parson 's wife",
 "T5\tCharacter_Line 303 407\twith the money that i get from the sale of these eggs i 'll buy myself a new dimity frock and a chip hat",
 "T6\tCharacter_Line 408 480\tand when i go to market wo n't all the young men come up and speak to me",
 'T7\tCharacter_Line 481 512\tpolly shaw will be that jealous',
 "T8\tCharacter_Line 513 530\tbut i do n't care",
 'T9\tCharacter_Line 531 582\ti shall just look at her and toss my head like this']

In [137]:

class Character:
    def __init__(self, varname, name):
        self.name = name
        self.varname = varname
        self.gender = ''
        self.age = ''
        
    def set_gender(self, gender):
        self.gender = gender
        
    def set_age(self, age):
        self.age = age
        
    def __str__(self):
        return __repr__(self)
    
    def __repr__(self):
        return "[{}:{}/{}/{}]".format(self.varname, self.name, self.gender, self.age)
    
    def to_json(self):
        json_dict = {"type":"cast_definition", "name":self.name}
        if self.gender != '':
            json.dict['gender'] = self.gender
        if self.age != '':
            json.dict['age'] = self.age
        
        return json_dict
    

In [138]:
characters = []

def find_character(var):
    for c in characters:
        if c.varname == var:
            return c

for tup in tups:
    #print(tup)
    
    groups = regexp_entity.findall(tup)
    if len(groups) > 0:
        if groups[0][1] == 'Character':
            characters.append(
                Character(groups[0][0].strip(), groups[0][2].strip())
            )
            
        continue
    groups = regexp_attribute.findall(tup)
    if len(groups) > 0:
        if groups[0][1] == 'Gender':
            c = find_character(groups[0][2].strip())
            c.set_gender(groups[0][3].strip().lower())
        elif groups[0][1] == 'Age':
            c = find_character(groups[0][2].strip())
            c.set_age(groups[0][3].strip().lower())            

In [125]:
regexp_attribute = re.compile('(?P<variable>[A-Z][0-9]+)\s+(?P<type>[A-Z][A-Za-z_]*)\s+(?P<target_variable>[A-Z][0-9]+)\s+(?P<value>[A-Za-z]+)')


In [139]:
for c in characters:
    print(c.to_json())


NameError: name 'json' is not defined